In [2]:
import random
from random import randint
import json
from datetime import datetime
from operator import add
import time
from itertools import chain
from itertools import combinations, groupby
from operator import add
from collections import Counter
import itertools
import json
import math
import random
import sys
import time
from functools import reduce
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import numpy as np

In [3]:
data_path = "/content/drive/MyDrive/DSCI553/project/data"

In [6]:
user_path = data_path+"/user.json"

# load the user and business features
user_features = {} 
with open(user_path) as file: 
    for l in file.readlines(): 
        data = json.loads(l)
        user_features[data["user_id"]]=[float(data["review_count"]),float(data["useful"]), float(data["average_stars"])]

business_path = data_path+"/business.json"
business_features = {}
with open(business_path) as file: 
    for l in file.readlines(): 
        data = json.loads(l)
        business_features[data["business_id"]] = [float(data["stars"]), float(data["review_count"]), float(data["is_open"])]
photo_path = data_path+"/photo.json"
photo_features = {}
with open(photo_path) as file: 
    for l in file.readlines(): 
        data = json.loads(l)
        business_id = data["business_id"]
        if business_id not in photo_features: 
            photo_features[business_id]=1.0
        else: 
            photo_features[business_id]+=1.0
# get the business/user pair and the ground truth
def getData(path): 
    train_path = path
    train = []
    with open (train_path) as file: 
        for l in file.readlines(): 
            data = l.rstrip().split(",")
            if data[0]!="user_id": 
                user_id = data[0]
                business_id = data[1]
                u_f = user_features[user_id]
                b_f = business_features[business_id]
                try: 
                    photo = [photo_features[business_id]]
                except: 
                    photo = [0.0]

                train.append(data+u_f+b_f+photo)
    return train

train_path = data_path+"/yelp_train.csv"
train = getData(train_path)
train_x = np.array([x[3:] for x in train])
train_y = np.array([float(x[2]) for x in train])
test_path = data_path+"/yelp_val.csv"
test = getData(test_path)
test_x = np.array([x[3:] for x in test])
test_y = np.array([float(x[2]) for x in test])

In [16]:
from sklearn.linear_model import LinearRegression
model = LinearRegression(normalize=True).fit(train_x, train_y)
predict = model.predict(test_x)
error = mean_squared_error(test_y, predict, squared=False)

In [17]:
print(error)

0.9871675283365483


In [10]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(train_x)
train_x_scaled = scaler.transform(train_x)
test_x_scaled = scaler.transform(test_x)

In [15]:
model_2 = LinearRegression().fit(train_x_scaled, train_y)
predict_2 = model_2.predict(test_x_scaled)
error_2 = mean_squared_error(test_y, predict_2, squared = False)
print(error_2)

0.9871675283365482


In [21]:
predict_2 = [round(x) for x in predict_2]
error_2 = mean_squared_error(test_y, predict, squared = False)
print(error_2)

0.9871675283365483


In [20]:
predict

array([3.81359553, 4.92491996, 4.852246  , ..., 3.72588599, 4.03155966,
       3.43204113])

In [26]:
from sklearn.neighbors import KNeighborsRegressor
min_error = float("Inf")
min_error_k = None
for k in range(1,50):
    knn = KNeighborsRegressor(n_neighbors = k).fit(train_x, train_y)
    pre = knn.predict(test_x)
    error = mean_squared_error(test_y, pre, squared = False)
    if error < min_error: 
        min_error = error
        min_error_k = k


In [24]:
error

1.1412129260736108